<a href="https://colab.research.google.com/github/engineereliab076/RANSFORMER_SPAM_MODEL_DETECTOR/blob/main/TRANSFORMER_SPAM_MODEL_DETECTOR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#download the transformers lib

!pip install transformers datasets torch scikit-learn

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
#import the dependencies
import numpy as np
import pandas as pd
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments
from transformers import Trainer, DataCollatorWithPadding

In [ ]:
checkpoint = "google-bert/bert-base-uncased"
max_length=128

In [ ]:
#load data
mail_data = load_dataset("ucirvine/sms_spam")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
print(mail_data)

DatasetDict({
    train: Dataset({
        features: ['sms', 'label'],
        num_rows: 5574
    })
})


In [ ]:

#train-test split

mail_data = mail_data["train"].train_test_split(test_size=0.2)

In [ ]:
print(mail_data)

DatasetDict({
    train: Dataset({
        features: ['sms', 'label'],
        num_rows: 4459
    })
    test: Dataset({
        features: ['sms', 'label'],
        num_rows: 1115
    })
})


DATA TOKENIZATION

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [ ]:
#set up the tokenization function
def tokenize_function(examples):
  return tokenizer(examples["sms"],
                   padding="max_length",
                   truncation=True,
                   max_length=max_length)

#map
tokenized_data = mail_data.map(tokenize_function, batched=True)

#remove sms and leave only tokenized data
tokenized_data = tokenized_data.remove_columns(["sms"])

#convert to py tensors
tokenized_data.set_format("torch")



Map:   0%|          | 0/4459 [00:00<?, ? examples/s]

Map:   0%|          | 0/1115 [00:00<?, ? examples/s]

In [ ]:
print(tokenized_data)

DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 4459
    })
    test: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1115
    })
})


MODEL TRAINING

In [ ]:
#load model

model = AutoModelForSequenceClassification.from_pretrained(checkpoint,
                                                              num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
#define training args

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    push_to_hub=True
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
#set data loader

train_dataset = tokenized_data["train"]
eval_dataset = tokenized_data["test"]

In [ ]:
#define data collator(it ensures all batches have the same shape by padding dynamically)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator
)

<ipython-input-15-4746ea316aac>:5: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
#train the model

trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: engineereliab076 (engineereliab076-rayat-bahra-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Epoch,Training Loss,Validation Loss
1,0.077000,0.017063
2,0.029400,0.021326
3,0.006100,0.022492


TrainOutput(global_step=1674, training_loss=0.03538985591350633, metrics={'train_runtime': 386.7628, 'train_samples_per_second': 34.587, 'train_steps_per_second': 4.328, 'total_flos': 879909146887680.0, 'train_loss': 0.03538985591350633, 'epoch': 3.0})

In [ ]:
metrics = trainer.evaluate()
print(metrics)

{'eval_loss': 0.022492272779345512, 'eval_runtime': 7.4524, 'eval_samples_per_second': 149.615, 'eval_steps_per_second': 18.786, 'epoch': 3.0}


MODEL EVALUTAION

In [ ]:
!pip install evaluate

In [ ]:
import evaluate
#load metrics
accuracy_metric = evaluate.load("accuracy")
precision_metric = evaluate.load("precision")
recall_metric = evaluate.load("recall")
f1_metric = evaluate.load("f1")

In [ ]:
#get the preds
predictions = trainer.predict(eval_dataset)

# Extract logits and convert them to label predictions
preds = predictions.predictions.argmax(axis=-1)
labels = predictions.label_ids

# Compute accuracy
accuracy = accuracy_metric.compute(predictions=preds, references=labels)

# Compute precision
precision = precision_metric.compute(predictions=preds, references=labels, average="binary")

# Compute recall
recall = recall_metric.compute(predictions=preds, references=labels, average="binary")

# Compute F1-score
f1 = f1_metric.compute(predictions=preds, references=labels, average="binary")

# Print results
print(f"Accuracy: {accuracy['accuracy']:.4f}")
print(f"Precision: {precision['precision']:.4f}")
print(f"Recall: {recall['recall']:.4f}")
print(f"F1-score: {f1['f1']:.4f}")

Accuracy: 0.9964
Precision: 0.9852
Recall: 0.9852
F1-score: 0.9852


In [ ]:
model.save_pretrained("spam_detector_model")
tokenizer.save_pretrained("spam_detector_model")


('spam_detector_model/tokenizer_config.json',
 'spam_detector_model/special_tokens_map.json',
 'spam_detector_model/vocab.txt',
 'spam_detector_model/added_tokens.json',
 'spam_detector_model/tokenizer.json')

In [ ]:
#predictions
def predict_spam(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)

    # Move inputs to the same device as the model
    for k, v in inputs.items():
        inputs[k] = v.to(model.device)

    outputs = model(**inputs)
    prediction = outputs.logits.argmax().item()  # Get the predicted label
    return "Spam" if prediction == 1 else "Not Spam"

# Test samples
message = """ 📧 Subject: Work From Home & Earn $500/day!
Body:

Hi,
We have an amazing remote job opportunity for you! No experience required, just a computer & internet. Earn up to $500/day easily from home.
Sign up now: fake-job-link.com
"""
print(predict_spam(message))

Spam


In [ ]:
trainer.push_to_hub()

CommitInfo(commit_url='https://huggingface.co/Engineereliab076/results/commit/dc3b6f62bed2b0ce5478d78adc8b33471c9e0cca', commit_message='End of training', commit_description='', oid='dc3b6f62bed2b0ce5478d78adc8b33471c9e0cca', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Engineereliab076/results', endpoint='https://huggingface.co', repo_type='model', repo_id='Engineereliab076/results'), pr_revision=None, pr_num=None)